In [1]:
import os
import pandas as pd
from qub_canvas_helper.assignments import CanvasAssignmentManager

# Canvas Login Details

In [2]:
# Where on the internet is canvas
canvas_domain = "https://qub.instructure.com"

# Access token is now in .env under CANVAS_API_TOKEN
access_token = os.getenv("CANVAS_API_TOKEN")

# Check this is set up properly
if not access_token:
    raise ValueError("Missing Canvas API token!")

# Configure Details

Specify the course. Some of the code will not work in a familiarisation environment as the has no sis_student_ids.

The course_id can be found in the url of the module: https://canvas.qub.ac.uk/courses/27862 (this is my familiarisation module)

In [3]:
course_id = 27862

Load the Excel file containing the list of students, which assignments they will have and when.

FORMAT: id (student ID) | name | "Date of assignment"

In [4]:
df_students = pd.read_excel("custom_assignments_test.xlsx")
#df_students['id'] = df_students['id'].astype('string') # stops an error later on
df_students.head()

,id,name,2024-12-23 00:00:00,2024-12-30 00:00:00
0,FA_Student3,Matt Damon,P1,P2
1,FA_Student2,Jodie Foster,P1,P2
2,FA_Student4,Tom Hanks,P1,P2
3,FA_Student5,Liam Neeson,P2,P1
4,FA_Student6,Notareal Person,P2,P1


# Start Talking to Canvas

Instantiate the class using the data provided above

In [5]:
assignment_manager = CanvasAssignmentManager(canvas_domain, access_token, course_id)

Return a dataframe of the students enrolled on canvas.

This is filtered for active students only.

Be patient, this can take a while for large modules.

In [6]:
assignment_manager.get_students_in_module()

,id,name,created_at,sortable_name,short_name,sis_user_id,integration_id
0,1491,Matt Damon,2018-05-09T10:06:55+01:00,"Damon, Matt",Matt Damon,FA_Student3,None
1,1490,Jodie Foster,2018-05-09T10:06:32+01:00,"Foster, Jodie",Jodie Foster,FA_Student2,None
2,1492,Tom Hanks,2018-05-09T10:07:15+01:00,"Hanks, Tom",Tom Hanks,FA_Student4,None
3,1493,Liam Neeson,2018-05-09T10:07:35+01:00,"Neeson, Liam",Liam Neeson,FA_Student5,None
4,1489,Kate Winslet,2018-05-09T10:06:07+01:00,"Winslet, Kate",Kate Winslet,FA_Student1,None


Return a dataframe of all the assignments in this module 

In [7]:
assignment_manager.get_assignments_in_module(published_status = "all") # 'all', 'published', or 'unpublished'

,id,name,due_at,points_possible,published
0,181313,Example Assignment,2024-10-31T23:59:59Z,100.0,True
1,203603,Assignment 1,2024-11-01T23:59:59Z,0.0,True
2,203604,Assignment 2,2024-11-01T23:59:59Z,0.0,True
3,204089,Post Lab 1,2024-11-08T23:59:59Z,0.0,True
4,204090,Post Lab 2,2024-11-08T23:59:59Z,0.0,False
5,204091,Post Lab 3,2024-11-08T23:59:59Z,0.0,False
6,204092,Post Lab 4,2024-11-08T23:59:59Z,0.0,False
7,204093,Post Lab 5,None,0.0,False


In [8]:
assignment_dict = {"P1":"203603", 
                   "P2":"203604"} 

Check that there are no missing or extra students on Canvas or in the Excel sheet. 

Works by comparing sis_student_id (Canvas) to the id specified in Excle. This must be the first column in the Excel sheet.

Be patient, this can take a while for large modules. It will not work for the familiarisation area.

In [9]:
assignment_manager.check_student_enrollment(df_students)

Students missing in Canvas:
- Notareal Person (ID: FA_Student6)
Students missing in Excel:
- Winslet, Kate (ID: FA_Student1)


# Start Making Changes on Canvas

Assign students to assignments based on the Excel sheet provided.

A check will first be made that there are no missing or extra students using the function above, this can be ignored if required.

Note: assign_students_to_assignments_familiarisation_area is specific to only the familiarisation area as the sis_use_IDs here are strings and not integers which becomes important outside the familiarisation area

In [10]:
assignment_manager.assign_students_to_assignments_familiarisation_area(df_students, assignment_dict, check_enrollment=False)

Assignment 203603 successfully assigned to student Matt Damon (1491) with due date 2024-12-23 00:00:00
Assignment 203604 successfully assigned to student Matt Damon (1491) with due date 2024-12-30 00:00:00
Assignment 203603 successfully assigned to student Jodie Foster (1490) with due date 2024-12-23 00:00:00
Assignment 203604 successfully assigned to student Jodie Foster (1490) with due date 2024-12-30 00:00:00
Assignment 203603 successfully assigned to student Tom Hanks (1492) with due date 2024-12-23 00:00:00
Assignment 203604 successfully assigned to student Tom Hanks (1492) with due date 2024-12-30 00:00:00
Assignment 203604 successfully assigned to student Liam Neeson (1493) with due date 2024-12-23 00:00:00
Assignment 203603 successfully assigned to student Liam Neeson (1493) with due date 2024-12-30 00:00:00
Failed to assign assignment 203604 to student Notareal Person (100). Response: {'errors': ['unknown student ids: ["100"]']}
Failed to assign assignment 203603 to student No

### Check that all students have been assigned

If the excel list was incorrect, some students may not have been assigned. We can take a look.

In [11]:
assignment_manager.check_student_assignment_completions(assignment_dict)

Student Kate Winslet (ID: 1489) is not assigned to P1 (ID: 203603).
Student Kate Winslet (ID: 1489) is not assigned to P2 (ID: 203604).


### Remove all assignments

If any mistakes have been made, then individual assignment to students can be removed, leaving the assignment as it was to begin with.

This can done for a single assignment or multiple assignments if entered as a list

In [12]:
assignment_manager.remove_student_assignments([203603,203604])

Successfully removed assignment 203603 for override 39594.
Successfully removed assignment 203603 for override 39596.
Successfully removed assignment 203603 for override 39598.
Successfully removed assignment 203603 for override 39601.
Successfully removed assignment 203604 for override 39595.
Successfully removed assignment 203604 for override 39597.
Successfully removed assignment 203604 for override 39599.
Successfully removed assignment 203604 for override 39600.
Completed removing overrides for assignments: [203603, 203604]
